# Project 2 - Classification
## Predict customers likely to respond to a marketing campaign
### This notebook uses the *campaign.xlsx* dataset

(c) Nuno António 2022 - Rev. 1.0

## Dataset description

- **AcceptedCmp1** - 1 if customer accepted the offer in the 1st campaign, 0 otherwise 
- **AcceptedCmp2** - 1 if customer accepted the offer in the 2nd campaign, 0 otherwise 
- **AcceptedCmp3** - 1 if customer accepted the offer in the 3rd campaign, 0 otherwise 
- **AcceptedCmp4** - 1 if customer accepted the offer in the 4th campaign, 0 otherwise 
- **AcceptedCmp5** - 1 if customer accepted the offer in the 5th campaign, 0 otherwise 
- **Response (target)** - 1 if customer accepted the offer in the last campaign, 0 otherwise 
- **Complain** - 1 if customer complained in the last 2 years
- **DtCustomer** - date of customer’s enrolment with the company
- **Education** - customer’s level of education
- **Marital** - customer’s marital status
- **Kidhome** - number of small children in customer’s household
- **Teenhome** - number of teenagers in customer’s household
- **Income** - customer’s yearly household income
- **MntFishProducts** - amount spent on fish products in the last 2 years
- **MntMeatProducts** - amount spent on meat products in the last 2 years
- **MntFruits** - amount spent on fruits products in the last 2 years
- **MntSweetProducts** - amount spent on sweet products in the last 2 years
- **MntWines** - amount spent on wine products in the last 2 years
- **MntGoldProds** - amount spent on gold products in the last 2 years
- **NumDealsPurchases** - number of purchases made with discount
- **NumCatalogPurchases** - number of purchases made using catalogue
- **NumStorePurchases** - number of purchases made directly in stores
- **NumWebPurchases** - number of purchases made through company’s web site
- **NumWebVisitsMonth** - number of visits to company’s web site in the last month
- **Recency** - number of days since the last purchase

[Link to the original dataset](https://github.com/ifood/ifood-data-business-analyst-test)  
[A complete solution](https://github.com/mgermy/project_model_cluster)  
[A smaller one](https://github.com/nailson/ifood-data-business-analyst-test)  
[Here you can find the discussion on kaggle](https://www.kaggle.com/datasets/jackdaoud/marketing-data?select=ifood_df.csv)

We can notice that we can build a monetary feature summing all the features related to monetary, and we can also built a frequency feature summing all the purchases features and then dividing by the dt_customer.  
Maybe can make sense after this engineering is done to make some percentage features, like the percentage spent in one category compared to the total and also the pecentage of purchases done compared to the total.

## Initializations and data loading

In [12]:
# Installing the necessary packages:
# Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns #visualizations

In [13]:
# Modelling packages

import collections
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from yellowbrick.classifier import ROCAUC
from yellowbrick.classifier import PrecisionRecallCurve
from sklearn.model_selection import GridSearchCV
from yellowbrick.model_selection import LearningCurve

In [14]:
# Formating that will be applied in all of the notebook
subPlots_Title_fontSize = 12
subPlots_xAxis_fontSize = 15
subPlots_yAxis_fontSize = 10
subPlots_label_fontSize = 12
heatmaps_text_fontSize = 12

plots_Title_fontSize = 14
plots_Title_textColour = 'black'

plots_Legend_fontSize = 12
plots_Legend_textColour = 'black'

# increase the number of columns to display
pd.set_option('display.max_columns', 500)

In [15]:
# Loading the dataset: 
ds = pd.read_parquet('oneHotEncoded.parquet.snappy')
ds.describe().T

,count,mean,std,min,25%,50%,75%,max
Income,1972.0,52139.991886,21470.778018,1730.0,35538.75,51741.5,68634.00,160803.0
Recency,1972.0,48.981744,28.947786,0.0,24.00,49.0,74.00,99.0
MntWines,1972.0,307.808824,338.723751,0.0,24.00,179.0,508.00,1493.0
MntFruits,1972.0,26.581136,39.980373,0.0,2.00,8.0,33.00,199.0
MntMeatProducts,1972.0,169.477688,228.297686,0.0,16.00,68.5,232.00,1725.0
MntFishProducts,1972.0,37.769777,55.014197,0.0,3.00,12.0,50.00,259.0
MntSweetProducts,1972.0,27.366633,41.572661,0.0,1.00,8.0,34.00,262.0
MntGoldProds,1972.0,44.034990,51.649919,0.0,9.00,25.0,57.00,321.0
NumDealsPurchases,1972.0,2.334178,1.923021,0.0,1.00,2.0,3.00,15.0
NumWebPurchases,1972.0,4.121704,2.757103,0.0,2.00,4.0,6.00,27.0


## KNN modeling

#### Data preparation

In [16]:
# Create a modeling dataset from the original dataset
X = ds.copy(deep=True)

In [17]:
X.head()

,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Age,Customer_Lifetime,TotalMnt,Education_2n Cycle,Education_Basic,Education_Graduation,Education_Master,Education_PhD,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Together,Marital_Status_Widow,Marital_Status_YOLO,Kidhome_0,Kidhome_1,Teenhome_0,Teenhome_1
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,58138,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,1,57,663.0,1617,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0
1,46344,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,0,60,113.0,27,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1
2,71613,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,0,49,312.0,776,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0
3,26646,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,0,30,139.0,53,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0
4,58293,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,0,33,161.0,422,0,0,0,0,1,0,1,0,0,0,0,0,1,1,0


In [18]:
# Create the Target
y = X['Response']

In [19]:
toDrop = ['Response', 'AvgWebPurchases']

The PCA will be used to reduce the number of variables used in the model:

In [20]:
from sklearn.decomposition import PCA

# Dataframe for scaling
tempDF = X.copy(deep=True)
tempDF.drop(columns=toDrop, inplace=True)

# Normalize training/test data
scaler = MinMaxScaler(feature_range=(0, 1))
tempDF_scaled = scaler.fit_transform(tempDF)

pca = PCA(n_components=0.9)
X = pca.fit_transform(tempDF_scaled)

KeyError: "['AvgWebPurchases'] not found in axis"

In [ ]:
# Split into train and test
X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=123)

### KNN

We will use the grid seach as the hyper tunning technique to find the best number of K. The seed is going from 2 by 2 because the number of neighbor have to be odd.

In [ ]:
param_grid_nb = {
    'n_neighbors': list(range(3,102,2))
}

grid = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=param_grid_nb, verbose=3, n_jobs=-1, scoring='roc_auc')

grid.fit(X_train_scaled, y_train)

In [ ]:
print(grid.best_params_)

In [ ]:
# Create object and train the model
classModel = KNeighborsClassifier(**grid.best_params_)
classModel.fit(X_train_scaled, y_train) # train the model (not training in reality)

In [ ]:
y_pred_train = classModel.predict(X_train_scaled) 
y_pred_test = classModel.predict(X_test_scaled) 

In [ ]:
# Function to create dataframe with metrics
def performanceMetricsDF(metricsObj, yTrain, yPredTrain, yTest, yPredTest):
  measures_list = ['ACCURACY','PRECISION', 'RECALL','F1 SCORE','AUC']
  train_results = [metricsObj.accuracy_score(yTrain, yPredTrain),
                metricsObj.precision_score(yTrain, yPredTrain),
                metricsObj.recall_score(yTrain, yPredTrain),
                metricsObj.f1_score(yTrain, yPredTrain),
                metricsObj.roc_auc_score(yTrain, yPredTrain)
                ]
  test_results = [metricsObj.accuracy_score(yTest, yPredTest),
               metricsObj.precision_score(yTest, yPredTest),
               metricsObj.recall_score(yTest, yPredTest),
               metricsObj.f1_score(yTest, yPredTest),
               metricsObj.roc_auc_score(yTest, yPredTest)
               ]
  resultsDF = pd.DataFrame({'Measure': measures_list, 'Train': train_results, 'Test':test_results})
  return(resultsDF)

In [ ]:
# Function to plot confusion matrix - Adapted from https://github.com/DTrimarchi10/confusion_matrix/blob/master/cf_matrix.py
def make_confusion_matrix(cf,
                          group_names=None,
                          categories='auto',
                          count=True,
                          percent=True,
                          cbar=True,
                          xyticks=True,
                          xyplotlabels=True,
                          sum_stats=True,
                          figsize=None,
                          cmap='Blues',
                          title=None):
    '''
    This function will make a pretty plot of an sklearn Confusion Matrix cm using a Seaborn heatmap visualization.
    Arguments
    ---------
    cf:            confusion matrix to be passed in
    group_names:   List of strings that represent the labels row by row to be shown in each square.
    categories:    List of strings containing the categories to be displayed on the x,y axis. Default is 'auto'
    count:         If True, show the raw number in the confusion matrix. Default is True.
    normalize:     If True, show the proportions for each category. Default is True.
    cbar:          If True, show the color bar. The cbar values are based off the values in the confusion matrix.
                   Default is True.
    xyticks:       If True, show x and y ticks. Default is True.
    xyplotlabels:  If True, show 'True Label' and 'Predicted Label' on the figure. Default is True.
    sum_stats:     If True, display summary statistics below the figure. Default is True.
    figsize:       Tuple representing the figure size. Default will be the matplotlib rcParams value.
    cmap:          Colormap of the values displayed from matplotlib.pyplot.cm. Default is 'Blues'
                   See http://matplotlib.org/examples/color/colormaps_reference.html
                   
    title:         Title for the heatmap. Default is None.
    '''


    # CODE TO GENERATE TEXT INSIDE EACH SQUARE
    blanks = ['' for i in range(cf.size)]

    if group_names and len(group_names)==cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks

    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks

    if percent:
        group_percentages = ["{0:.2%}".format(value) for value in cf.flatten()/np.sum(cf)]
    else:
        group_percentages = blanks

    box_labels = [f"{v1}{v2}{v3}".strip() for v1, v2, v3 in zip(group_labels,group_counts,group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])


    # CODE TO GENERATE SUMMARY STATISTICS & TEXT FOR SUMMARY STATS
    if sum_stats:
        #Accuracy is sum of diagonal divided by total observations
        accuracy  = np.trace(cf) / float(np.sum(cf))

        #if it is a binary confusion matrix, show some more stats
        if len(cf)==2:
            #Metrics for Binary Confusion Matrices
            precision = cf[1,1] / sum(cf[:,1])
            recall    = cf[1,1] / sum(cf[1,:])
            f1_score  = 2*precision*recall / (precision + recall)
            stats_text = "\n\nAccuracy={:0.3f}\nPrecision={:0.3f}\nRecall={:0.3f}\nF1 Score={:0.3f}".format(
                accuracy,precision,recall,f1_score)
        else:
            stats_text = "\n\nAccuracy={:0.3f}".format(accuracy)
    else:
        stats_text = ""


    # SET FIGURE PARAMETERS ACCORDING TO OTHER ARGUMENTS
    if figsize==None:
        #Get default figure size if not set
        figsize = plt.rcParams.get('figure.figsize')

    if xyticks==False:
        #Do not show categories if xyticks is False
        categories=False


    # MAKE THE HEATMAP VISUALIZATION
    plt.figure(figsize=figsize)
    ax = sns.heatmap(cf,annot=box_labels, fmt="",cmap=cmap,cbar=cbar,xticklabels=categories,yticklabels=categories)

    if xyplotlabels:
        plt.ylabel('True label')
        plt.xlabel('Predicted label' + stats_text)
    else:
        plt.xlabel(stats_text)
    
    if title:
        plt.title(title)

In [ ]:
# Show the confusion matrix
cf = metrics.confusion_matrix(y_test,y_pred_test)
labels = ['True Neg','False Pos','False Neg','True Pos']
categories = ['0', '1']
make_confusion_matrix(cf, 
                      group_names=labels,
                      categories=categories, 
                      cmap='Blues')

In [ ]:
# Show performance results
resultsDF = performanceMetricsDF(metrics, y_train, y_pred_train, y_test, y_pred_test)
resultsDF

In [ ]:
# ROC curve
visualizer = ROCAUC(classModel, classes=['0','1'])
visualizer.fit(X_train_scaled, y_train)
visualizer.score(X_test_scaled, y_test)
visualizer.show()

In [ ]:
# Precison-Recall curve
visualizer = PrecisionRecallCurve(classModel, classes=['0','1'])
visualizer.fit(X_train_scaled, y_train)
visualizer.score(X_test_scaled, y_test)
visualizer.show()

In [ ]:
# Plot the learning curve
cv = 10
visualizer = LearningCurve(estimator=classModel, cv=cv, scoring='roc_auc', n_jobs=-1, random_state=123)
visualizer.fit(X_train_scaled, y_train)
visualizer.show()

### Balancing dataset
#### Oversampling

In [ ]:
# Display target balance in the training dataset
print(collections.Counter(y_train))
fig, ax = plt.subplots(figsize=(6,4))
sns.countplot(x="y", data=pd.DataFrame(data={'y':y_train}), ax=ax)

Knn prefers to have a sampling_stategy equal to 1, that was expected given the nature of the algorithm.

In [ ]:
# Import package
from imblearn.over_sampling import SMOTE

# Applyting SMOTE to generare new instances (oversampling)
sm = SMOTE(random_state=123, sampling_strategy=1.0)
X_train_scaled2, y_train2 = sm.fit_resample(X_train_scaled, y_train)

# Display target balance in the training dataset
print(collections.Counter(y_train2))
fig, ax = plt.subplots(figsize=(6,4))
sns.countplot(x="y", data=pd.DataFrame(data={'y':y_train2}), ax=ax)

Considering that we oversampled the data above, it will be necessary to hyper tune it again using grid search to find the best K:

In [ ]:
param_grid_nb = {
    'n_neighbors': list(range(3,102,2))
}

grid = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=param_grid_nb, verbose=3, n_jobs=-1, scoring='roc_auc')

grid.fit(X_train_scaled2, y_train2)

In [ ]:
# Create object and train the model
classModel = KNeighborsClassifier(**grid.best_params_)
classModel.fit(X_train_scaled2, y_train2) # train the model (not training in reality)

In [ ]:
y_pred_train = classModel.predict(X_train_scaled2) 
y_pred_test = classModel.predict(X_test_scaled) 

In [ ]:
# Show the confusion matrix
cf = metrics.confusion_matrix(y_test,y_pred_test)
labels = ['True Neg','False Pos','False Neg','True Pos']
categories = ['0', '1']
make_confusion_matrix(cf, 
                      group_names=labels,
                      categories=categories, 
                      cmap='Blues')


In [ ]:
# Show performance results
resultsDF = performanceMetricsDF(metrics, y_train2, y_pred_train, y_test, y_pred_test)
resultsDF

In [ ]:
# ROC curve
visualizer = ROCAUC(classModel, classes=['0','1'])
visualizer.fit(X_train_scaled2, y_train)
visualizer.score(X_test_scaled, y_test)
visualizer.show()

In [ ]:
# Plot the learning curve
cv = 10
visualizer = LearningCurve(estimator=classModel, cv=cv, scoring='roc_auc', n_jobs=-1, random_state=123)
visualizer.fit(X_train_scaled2, y_train2)
visualizer.show()